In [1]:
import udma
import numpy as np
import fxpmath as fx
import matplotlib.pyplot as plt
from tqdm import trange

In [2]:
def dec2fixed(dec, left, right):
    binary = np.binary_repr(dec, width = (left - right + 1))
    fixed = 0
    for i,j in enumerate(np.arange(left, right - 1, -1)):
        if j<0:
            fixed += int(binary[i]) / (2**(-j))
        else:
            fixed += int(binary[i]) * (2**(j))
    return fixed

In [3]:
print(dec2fixed(8103, 2, -13))

0.9891357421875


In [40]:
x1 = (1 - 0.999767780303955)
x2 = dec2fixed(974, 4, -11)
print(x1/x2)

0.0004882812500002334


In [26]:
cb = udma.UDMA_CLASS('192.168.1.10', 7)
cb.connect()
cb.log(0)

[(5,), 0]

In [42]:
cb.write_reg(0, 1)
cb.write_reg(0, 0)

[(1, 0), 0]

In [46]:
for i in trange(1):
    p0 = cb.read_reg(0)[1][0]
    p1 = cb.read_reg(1)[1][0]     
    x = np.random.randint(0,32767)  
    cb.write_fifo(1, [x])
    while (cb.read_reg(5)[1][0] & 1 == 0):
        pass
    px0 = dec2fixed(p0, 2, -13)
    px1 = dec2fixed(p1, 2, -13)
    rw = dec2fixed(x, 5, -11) * 0.0001220703125
    ts1 = 1 - rw
    ts2 = rw
    ps0 = px0 * ts1 + ts2
    ps1 = px1 * ts1
    th1 = dec2fixed(cb.read_reg(2)[1][0], 2, -13)
    th2 = dec2fixed(cb.read_reg(3)[1][0], -9, -24)
    ph0 = dec2fixed(cb.read_reg(0)[1][0], 2, -13)
    ph1 = dec2fixed(cb.read_reg(1)[1][0], 2, -13)
    print("SOFTWARE -> {} = {} * {} + {} || {} = {} * {}".format(ps0, px0, ts1, ts2, ps1, px0, ts1))
    print("HARDWARE -> {} = {} * {} + {} || {} = {} * {}".format(ph0, px0, th1, th2, ph1, px0, th1))

100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

SOFTWARE -> 0.004195635534415487 = 0.0032958984375 * 0.9990972876548767 + 0.000902712345123291 || 0.015488935001485515 = 0.0032958984375 * 0.9990972876548767
HARDWARE -> 0.0042724609375 = 0.0032958984375 * 0.9990234375 + 0.000902712345123291 || 0.0155029296875 = 0.0032958984375 * 0.9990234375


In [44]:
ts2 / th2

1.0